<a href="https://colab.research.google.com/github/Damephena/QA-NLP/blob/main/finals_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [ ]:
!pip install nbconvert

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import torch
import sklearn
import wandb
import json
import logging
import os
from statistics import mean, mode

import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# Dataset Preparation

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

In [4]:
with open('/content/drive/MyDrive/ColabNotebooks/train-v2.0.json', 'r') as f:
    data = json.load(f)

In [5]:
data = [item for topic in data['data'] for item in topic['paragraphs'] ]

In [6]:
new_eval_data = data[18000:]

In [7]:
train_data = data[:10000]

# Model Configuration

Setup wandb.ai run

In [ ]:
config = {
    'learning_rate': 1e-4,
    'architecture': 'transformer',
    'model': 'distilbert-base-uncased',
    'training_set': 10000,
    'evaluation_set': 1035,
    'num_train_epochs': 30,
    'max_seq_length': 128,
    'doc_stride': 384,
    'overwrite_output_dir': True,
    'reprocess_input_data': True,
    'train_batch_size': 32,
    'eval_batch_size': 16,
    'evaluate_during_training': False,
    'max_query_length': 128,
    'max_answer_length': 250,
    'cache_dir': '/content/drive/MyDrive/ColabNotebooks/finals-2/cache_dir',
    'output_dir': '/content/drive/MyDrive/ColabNotebooks/finals-2/outputs',
    'do_lower_case': True,
}

# start a project
wandb.init(
    project='finals-2',
    entity='Ifenna',
    tags=['32-train-batch-size', 'finals-2', '15-epoch'],
    config=config
)

In [ ]:
# ddd45a5bff584acfead149a3bf38206cf3b14c24
config['learning_rate']

0.0001

In [ ]:
train_args = {
    'learning_rate': wandb.config['learning_rate'],
    'num_train_epochs': wandb.config['num_train_epochs'],
    'max_seq_length': wandb.config.max_seq_length,
    'doc_stride': wandb.config.doc_stride,
    'overwrite_output_dir': wandb.config.overwrite_output_dir,
    'reprocess_input_data': wandb.config.reprocess_input_data,
    'train_batch_size': wandb.config.train_batch_size,
    'eval_batch_size': wandb.config.eval_batch_size,
    'evaluate_during_training': wandb.config.evaluate_during_training,
    'max_query_length': wandb.config.max_query_length,
    'max_answer_length': wandb.config.max_answer_length,
    'cache_dir': wandb.config.cache_dir,
    'output_dir': wandb.config.output_dir,
    'do_lower_case': wandb.config.do_lower_case,
    'fp16': False,
    'wandb_project': 'finals-2',
    'wandb_kwargs': {'entity': 'Ifenna'}
}

# train_args = {
#     'learning_rate': config['learning_rate'],
#     'num_train_epochs': config['num_train_epochs'],
#     'max_seq_length': config['max_seq_length'],
#     'doc_stride': config['doc_stride'],
#     'overwrite_output_dir': config['overwrite_output_dir'],
#     'reprocess_input_data': config['reprocess_input_data'],
#     'train_batch_size': config['train_batch_size'],
#     'eval_batch_size': config['eval_batch_size'],
#     'evaluate_during_training': config['evaluate_during_training'],
#     'max_query_length': config['max_query_length'],
#     'max_answer_length': config['max_answer_length'],
#     'cache_dir': config['cache_dir'],
#     'output_dir': config['output_dir'],
#     'do_lower_case': config['do_lower_case'],
#     'fp16': False,
#     'wandb_project': 'finals-2',
#     'wandb_kwargs': {'entity': 'Ifenna'}
# }

# Train Model

In [8]:
cuda_available = torch.cuda.is_available()
model = QuestionAnsweringModel(
    'distilbert',
    # 'distilbert-base-uncased',
    '/content/drive/MyDrive/ColabNotebooks/finals-2/outputs/checkpoint-official-epoch-15',
    # args=train_args,
    # sweep_config=wandb.config,
    use_cuda=cuda_available
)

In [ ]:
model.train_model(train_data)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features: 100%|██████████| 65488/65488 [05:15<00:00, 207.32it/s]

add example index and unique id: 100%|██████████| 65488/65488 [00:00<00:00, 805104.16it/s]


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model:   Continuing training from checkpoint, will skip to saved global_step
INFO:simpletransformers.question_answering.question_answering_model:   Continuing training from epoch 15
INFO:simpletransformers.question_answering.question_answering_model:   Continuing training from global step 30930
INFO:simpletransformers.question_answering.question_answering_model:   Will skip the first 0 steps in the current epoch


Running Epoch 0 of 30:   0%|          | 0/2062 [00:00<?, ?it/s]

Running Epoch 1 of 30:   0%|          | 0/2062 [00:00<?, ?it/s]

...

Running Epoch 12 of 30:   0%|          | 0/2062 [00:00<?, ?it/s]

Running Epoch 13 of 30:   0%|          | 0/2062 [00:00<?, ?it/s]

Running Epoch 14 of 30:   0%|          | 0/2062 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Training of distilbert model complete. Saved to /content/drive/MyDrive/ColabNotebooks/finals-2/outputs.


(61860, 0.0533937148350514)